In [1]:
import os
import tensorflow as tf
import tensorflow_recommenders as tfrs
import json
import pandas as pd
from os import getenv
#from keras import models
from firebase_admin import credentials, firestore, initialize_app

In [2]:
cred = credentials.Certificate("key.json")
default_app = initialize_app(cred)
db = firestore.client()

In [3]:
class User(object):
    def __init__(self, USER_ID, Nama, Gender, Age, Tipe_Masalah):
        self.USER_ID = USER_ID
        self.Nama = Nama
        self.Gender = Gender
        self.Age = Age
        self.Tipe_Masalah = Tipe_Masalah

    @staticmethod
    def from_json_do_not_use(source):
        """
        Source is expected from pure json string
        """
        return source

    @staticmethod
    def from_dict(source):
        # [START_EXCLUDE]
        user = User(source[u'USER_ID'], source[u'Nama'], source[u'Gender'], source[u'Age'], source[u'Tipe_Masalah'])

        if u'USER_ID' in source:
            user.USER_ID = source[u'USER_ID']

        if u'Nama' in source:
            user.Nama = source[u'Nama']

        if u'Gender' in source:
            user.Gender = source[u'Gender']

        if u'Age' in source:
            user.Age = source[u'Age']

        if u'Tipe_Masalah' in source:
            user.Tipe_Masalah = source[u'Tipe_Masalah']

        return user
        # [END_EXCLUDE]

    def to_dict(self):
        # [START_EXCLUDE]
        dest = {
            u'USER_ID': self.USER_ID,
            u'Nama': self.Nama,
            u'Gender': self.Gender,
            u'Age': self.Age,
            u'Tipe_Masalah': self.Tipe_Masalah
        }

        if self.USER_ID:
            dest[u'USER_ID'] = self.USER_ID

        if self.Nama:
            dest[u'Nama'] = self.Nama

        if self.Gender:
            dest[u'Gender'] = self.Gender

        if self.Age:
            dest[u'Age'] = self.Age

        if self.Tipe_Masalah:
            dest[u'Tipe_Masalah'] = self.Tipe_Masalah

        return dest
        # [END_EXCLUDE]

    
    def __repr__(self):
        return (
            f'City(\
                USER_ID={self.USER_ID}, \
                Nama={self.Nama}, \
                Age={self.Age}, \
                Gender={self.Gender}, \
                Tipe_Masalah={self.Tipe_Masalah}\
            )'
        )

In [4]:
zidan = User(1, "zidan", "pria", 43, "Depresi")

In [5]:
zidan

City(                USER_ID=1,                 Nama=zidan,                 Age=43,                 Gender=pria,                 Tipe_Masalah=Depresi            )

In [6]:
type(zidan)

__main__.User

In [7]:
type(zidan.to_dict())

dict

In [8]:
import pathlib
import os

user_file = "\\Data\\Processed User Data.json"
caregiver_file = "\\Data\\Processed Caregiver Data.json"

path = pathlib.Path().resolve()
parent_path = path.parent.absolute()

user_path = str(parent_path) + user_file
caregiver_ = str(parent_path) + caregiver_file

In [9]:
"""
with open(user_path) as file:
    data = json.load(file)
    db_user_placeholder = db.collection(u'User_Data_Placeholder')
    for user_dict in data:
        user_dict['Nama'] = 'placeholder_nama'
        user_dict['Text'] = 'Placeholder_text'
        db_user_placeholder.add(user_dict)
"""

"\nwith open(user_path) as file:\n    data = json.load(file)\n    db_user_placeholder = db.collection(u'User_Data_Placeholder')\n    for user_dict in data:\n        user_dict['Nama'] = 'placeholder_nama'\n        user_dict['Text'] = 'Placeholder_text'\n        db_user_placeholder.add(user_dict)\n"

In [10]:
data = db.collection(u'user_pref').get()


In [11]:
print(data[0].to_dict())

{'problems': 'Gangguan-kecemasan Depresi', 'text': 'Placeholder_text', 'age': 27, 'gender': 'Laki-laki', 'name': 'dhiva'}


In [12]:
def convert_snapshot(data, desired_key = None):
    """
    this extract desired keys from list of dicts from snapshot
    """
    data_dictionary = map(lambda x: x.to_dict(), data)
    if (desired_key is not None):
        data_dictionary = map(lambda x: dict((k, x[k]) for k in desired_key if k in x), list(data_dictionary))

    return list(data_dictionary)

In [13]:
new_data = convert_snapshot(data, ["gender","problems","age","text"])

In [14]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    """
    A utility method to create a tf.data dataset from a Pandas Dataframe
    """
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices(dict(dataframe))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [15]:
def convert_categorical_data(df, col, is_caregiver = False):
    """
    Convert categorical data that splited by space to their own column in pandas dataframe , making them like one hot encoded
    """
    import numpy as np
    ### Join every string in every row, split the result, pull out the unique values.
    genres = np.unique(' '.join(df[col]).split(' '))
    ### Drop 'NA'
    genres = np.delete(genres, np.where(genres == ''))
    if(not is_caregiver):
        for genre in genres:
            df[genre] = df[col].str.contains(genre).astype('int')
    else:
        for genre in genres:
            df['Caregiver-'+genre] = df[col].str.contains(genre).astype('int')
    df.drop(col, axis=1, inplace=True)

In [16]:
def fill_non_existent_column(df, is_caregiver = False):
    """
    This fill the df with unique problems column i
    """
    if is_caregiver is False:
         new_column = ['ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
    else:
        new_column = ['Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-kecemasan', 'Caregiver-Gangguan-makan','Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia']

    for col in new_column:
        if(col in df.columns):
            pass
        else:
            df[col] = 0
    
    

In [17]:
def change_column_name(df, old_name,new_name):
    """
    replace list of column name in old_name to new colum name in new_name
    the order of old_name and new_name need to be exact
    """
    res = {old_name[i]: new_name[i] for i in range(len(new_name))}
    df.rename(columns = res, inplace = True)
    
    

In [18]:
def desired_keys(dictionary , desired_key):
    if (desired_key is not None):
        dictionary = map(lambda x: dict((k, x[k]) for k in desired_key if k in x), list(dictionary))
    return list(dictionary)

In [19]:
#ds_kalimat = tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(new_data).to_dict(orient="list"))

In [20]:
user_df = pd.DataFrame.from_dict(new_data)

In [21]:
user_df.head(4)

,gender,problems,age,text
0,Laki-laki,Gangguan-kecemasan Depresi,27,Placeholder_text


In [22]:
convert_categorical_data(user_df, col='problems')

In [23]:
user_df.head(4)

,gender,age,text,Depresi,Gangguan-kecemasan
0,Laki-laki,27,Placeholder_text,1,1


In [24]:
change_column_name(user_df, ['gender','age'], ['Gender','Age'])

In [25]:
user_df.head(4)

,Gender,Age,text,Depresi,Gangguan-kecemasan
0,Laki-laki,27,Placeholder_text,1,1


In [26]:
user_df.columns

Index(['Gender', 'Age', 'text', 'Depresi', 'Gangguan-kecemasan'], dtype='object')

In [27]:
 fill_non_existent_column(user_df)

In [28]:
user_df.head(4)


,Gender,Age,text,Depresi,Gangguan-kecemasan,ADHD-Hiperaktif-dan-kurang-fokus,Gangguan-makan,Gangguan-stres-pascatrauma,Skizofrenia
0,Laki-laki,27,Placeholder_text,1,1,0,0,0,0


In [29]:
user_ds = df_to_dataset(user_df)

In [30]:
#caregiver_query

In [31]:
data = db.collection(u'chat_rooms').get()

In [32]:
from datetime import date
 
def age(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

In [33]:
import flatdict

In [34]:

def Merge(dict1, dict2):
    """
    Python code to merge dict
    """
    res = {**dict1, **dict2}
    return res
    

def unpack_caregiver_dict(dictionary):
    return 0

In [50]:
def unpack_caregiver_snapshot(data):
    desired_keys = ["problems", "birthday", "gender","caregiver_id","text"]
    #this unpack the value of snapshot list and turn the snapshot into dictionary also add aditional key for caregiver_id
    caregiver_dict_unpacked = map(lambda feature: Merge(feature.to_dict(), {'caregiver_id':feature.id}) , data)
    
    #this unpack the snapshot in 'caregiver' snapshot and add them back to main dictionary
    caregiver_dict = map(lambda feature: Merge(feature , feature.pop('caregiver').get().to_dict()) , caregiver_dict_unpacked )
    
    #this seperate desired keys from unused keys
    caregiver_dict = map(lambda x: dict((k, x[k]) for k in desired_keys if k in x), caregiver_dict)
    
    #this convert birthday stamp into interger age
    caregiver_dict = map(lambda feature: Merge(feature , {'age':age(feature.pop('birthday'))}), caregiver_dict)
    
    return list(caregiver_dict)

In [51]:
caregiver_dict = unpack_caregiver_snapshot(data)

In [52]:
caregiver_dict

[{'problems': 'Skizofrenia Depresi',
  'gender': 'Pria',
  'caregiver_id': '1294NxDFF60ozpXtMLzj',
  'text': 'cxcxc',
  'age': 21},
 {'problems': 'Gangguan-makan ADHD-Hiperaktif-dan-kurang-fokus',
  'gender': 'Pria',
  'caregiver_id': 'HUDwzgELY1OEzwieoHXK',
  'text': 'wrre',
  'age': 21},
 {'problems': 'Gangguan-makan Gangguan-stres-pascatrauma',
  'gender': 'Pria',
  'caregiver_id': 'UQr9WtwBTKhClEoe1AsS',
  'text': 'w344',
  'age': 21},
 {'problems': 'Gangguan-stres-pascatrauma',
  'gender': 'Pria',
  'caregiver_id': 'bYyoJYfth1fKqWBNI86A',
  'text': 'trtr',
  'age': 21},
 {'problems': 'Gangguan-stres-pascatrauma',
  'gender': 'Pria',
  'caregiver_id': 'hYwGH1IJzqwiDChn3zUU',
  'text': 'sf',
  'age': 21}]

In [53]:
def convert_caregiver_dictList_to_df(dictionary):
    # Creates DataFrame. 
    df = pd.DataFrame(dictionary) 
    #change column name to feature according to the model
    change_column_name(df, ['problems','gender','caregiver_id','age'],['Caregiver_Tipe_Masalah','Caregiver_Gender','CAREGIVER_ID','Caregiver_Age'])
    #convert categorical data into their own column
    convert_categorical_data(df, "Caregiver_Tipe_Masalah", is_caregiver = True)
    #fill missing column with default value
    fill_non_existent_column(df, is_caregiver = True)
    return df

In [54]:
# Creates DataFrame. 
df = convert_caregiver_dictList_to_df(caregiver_dict) 

In [55]:
caregiver_ds = df_to_dataset(df)

In [56]:
for feature_batch in caregiver_ds:
  print('Every feature:', list(feature_batch.keys()))

Every feature: ['Caregiver_Gender', 'CAREGIVER_ID', 'text', 'Caregiver_Age', 'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus', 'Caregiver-Depresi', 'Caregiver-Gangguan-makan', 'Caregiver-Gangguan-stres-pascatrauma', 'Caregiver-Skizofrenia', 'Caregiver-Gangguan-kecemasan']


In [57]:
df

,Caregiver_Gender,CAREGIVER_ID,text,Caregiver_Age,Caregiver-ADHD-Hiperaktif-dan-kurang-fokus,Caregiver-Depresi,Caregiver-Gangguan-makan,Caregiver-Gangguan-stres-pascatrauma,Caregiver-Skizofrenia,Caregiver-Gangguan-kecemasan
0,Pria,1294NxDFF60ozpXtMLzj,cxcxc,21,0,1,0,0,1,0
1,Pria,HUDwzgELY1OEzwieoHXK,wrre,21,1,0,1,0,0,0
2,Pria,UQr9WtwBTKhClEoe1AsS,w344,21,0,0,1,1,0,0
3,Pria,bYyoJYfth1fKqWBNI86A,trtr,21,0,0,0,1,0,0
4,Pria,hYwGH1IJzqwiDChn3zUU,sf,21,0,0,0,1,0,0


In [58]:
caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_CAREGIVER_MODEL(features)))

<MapDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 32), dtype=tf.float32, name=None))>

In [59]:
caregiver_index = df.to_dict('records')

In [60]:
caregiver_index

[{'Caregiver_Gender': 'Pria',
  'CAREGIVER_ID': '1294NxDFF60ozpXtMLzj',
  'text': 'cxcxc',
  'Caregiver_Age': 21,
  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus': 0,
  'Caregiver-Depresi': 1,
  'Caregiver-Gangguan-makan': 0,
  'Caregiver-Gangguan-stres-pascatrauma': 0,
  'Caregiver-Skizofrenia': 1,
  'Caregiver-Gangguan-kecemasan': 0},
 {'Caregiver_Gender': 'Pria',
  'CAREGIVER_ID': 'HUDwzgELY1OEzwieoHXK',
  'text': 'wrre',
  'Caregiver_Age': 21,
  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus': 1,
  'Caregiver-Depresi': 0,
  'Caregiver-Gangguan-makan': 1,
  'Caregiver-Gangguan-stres-pascatrauma': 0,
  'Caregiver-Skizofrenia': 0,
  'Caregiver-Gangguan-kecemasan': 0},
 {'Caregiver_Gender': 'Pria',
  'CAREGIVER_ID': 'UQr9WtwBTKhClEoe1AsS',
  'text': 'w344',
  'Caregiver_Age': 21,
  'Caregiver-ADHD-Hiperaktif-dan-kurang-fokus': 0,
  'Caregiver-Depresi': 0,
  'Caregiver-Gangguan-makan': 1,
  'Caregiver-Gangguan-stres-pascatrauma': 1,
  'Caregiver-Skizofrenia': 0,
  'Caregiver-Gangguan-kecemas

In [ ]:
caregiver_index = map(lambda x: dict((k, [x[k]]) for k in x), caregiver_index)

In [ ]:
list(caregiver_index)

In [45]:
MAIN_TEXT_MODEL = tf.saved_model.load('saved_model_v1/text_query_v1')
MAIN_USER_MODEL =   tf.keras.models.load_model('saved_manual_model/user_query_v1')
MAIN_CAREGIVER_MODEL =  tf.keras.models.load_model('saved_manual_model/caregiver_query_v1')

In [61]:
caregiver_index = caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_CAREGIVER_MODEL(features)))

In [62]:
brute_force = tfrs.layers.factorized_top_k.BruteForce(MAIN_USER_MODEL)
brute_force.index_from_dataset(  
       caregiver_ds.map(lambda features: (features['CAREGIVER_ID'], MAIN_CAREGIVER_MODEL(features)))
)





In [63]:
a =['Gender', 'Age', 'ADHD-Hiperaktif-dan-kurang-fokus', 'Depresi', 'Gangguan-kecemasan', 'Gangguan-makan','Gangguan-stres-pascatrauma', 'Skizofrenia']
value = ['Pria', 30, 0, 1, 0, 0, 1, 0]
dictionary_Test = dict((el,([value[ix]])) for ix,el in enumerate(a))
print(dictionary_Test)

{'Gender': ['Pria'], 'Age': [30], 'ADHD-Hiperaktif-dan-kurang-fokus': [0], 'Depresi': [1], 'Gangguan-kecemasan': [0], 'Gangguan-makan': [0], 'Gangguan-stres-pascatrauma': [1], 'Skizofrenia': [0]}


In [64]:

a, titles = brute_force(dictionary_Test, k=2)
print(f"Top recommendations: {titles[0]}")

Top recommendations: [b'bYyoJYfth1fKqWBNI86A' b'hYwGH1IJzqwiDChn3zUU']
